In [86]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,accuracy_score,classification_report
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas

In [87]:
# CHOOSE DATASET

# Binary classification dataset
data = datasets.load_breast_cancer(as_frame=True)

# Regression dataset
data = datasets.fetch_openml(name="boston",version=1, as_frame=True) 

X = data.data.values
y = data.target.values 
X.shape

(506, 13)

In [88]:
#train test spliting
test_size=0.2
Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=test_size, random_state=42)

In [89]:
# Standardize features
scaler=StandardScaler()
Xtr= scaler.fit_transform(Xtr)
Xte= scaler.transform(Xte)

In [90]:
class MLP(nn.Module):
    def __init__(self, input_size, output_size=1, dropout_prob=0.5):
        super(MLP, self).__init__()
        
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 64)
        self.out = nn.Linear(64, output_size)
        
        self.dropout = nn.Dropout(p=dropout_prob)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        
        x = self.out(x)
        return x

In [91]:
num_epochs=100
lr=0.0005
dropout=0.1
batch_size=64

In [92]:
Xtr = torch.tensor(Xtr, dtype=torch.float32)
ytr = torch.tensor(ytr, dtype=torch.float32)
Xte = torch.tensor(Xte, dtype=torch.float32)
yte = torch.tensor(yte, dtype=torch.float32)

# Wrap Xtr and ytr into a dataset
train_dataset = TensorDataset(Xtr, ytr)

# Create DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [93]:
# Model, Loss, Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MLP(input_size=Xtr.shape[1], dropout_prob=dropout).to(device)
criterion = nn.BCEWithLogitsLoss()  # for binary classification
criterion = nn.MSELoss() #for regression
optimizer = optim.Adam(model.parameters(), lr=lr)

In [94]:
# Training loop
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0.0

    for batch_x, batch_y in train_dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        logits = model(batch_x)
        loss = criterion(logits, batch_y.view(-1, 1))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

Epoch [1/100], Loss: 592.8591
Epoch [2/100], Loss: 594.5111
Epoch [3/100], Loss: 604.0402
Epoch [4/100], Loss: 594.9576
Epoch [5/100], Loss: 565.4299
Epoch [6/100], Loss: 533.9412
Epoch [7/100], Loss: 509.8226
Epoch [8/100], Loss: 411.6915
Epoch [9/100], Loss: 311.1153
Epoch [10/100], Loss: 192.7836
Epoch [11/100], Loss: 113.6246
Epoch [12/100], Loss: 99.5156
Epoch [13/100], Loss: 83.5621
Epoch [14/100], Loss: 71.5500
Epoch [15/100], Loss: 65.8479
Epoch [16/100], Loss: 50.8383
Epoch [17/100], Loss: 45.5812
Epoch [18/100], Loss: 39.5605
Epoch [19/100], Loss: 37.8969
Epoch [20/100], Loss: 36.1531
Epoch [21/100], Loss: 30.3478
Epoch [22/100], Loss: 33.0839
Epoch [23/100], Loss: 30.4297
Epoch [24/100], Loss: 35.3256
Epoch [25/100], Loss: 30.1855
Epoch [26/100], Loss: 30.2523
Epoch [27/100], Loss: 27.1450
Epoch [28/100], Loss: 30.3467
Epoch [29/100], Loss: 27.5267
Epoch [30/100], Loss: 28.8545
Epoch [31/100], Loss: 28.0572
Epoch [32/100], Loss: 28.8432
Epoch [33/100], Loss: 25.0516
Epoch [3

In [95]:
y_pred=model(Xte)
#print(f'ACC:{accuracy_score(yte.detach().numpy(),y_pred.detach().numpy()>0.5)}') #classification
print(f'MSE:{mean_squared_error(yte.detach().numpy(),y_pred.detach().numpy())}') #regression

MSE:20.853485107421875
